In [ ]:
import pandas as pd
import numpy as np
from common import *

In [ ]:
start_date = pd.Timestamp('2025-06-01')
today = pd.Timestamp.today()
eth_ticker = 'ETH-USD'
bitmine_ticker = 'BMNR'

In [ ]:
bitmine_holdings = pd.DataFrame({
    'date': [pd.Timestamp('20250630'), pd.Timestamp('20250708'), pd.Timestamp('20250805'), pd.Timestamp('20250811')],
    'eth': [0, 566776, 833137, 1150000]
})

In [ ]:
bitmine_holdings

In [ ]:
eth_px_data = get_historical_close_px(eth_ticker, start_date=start_date, end_date=today)
assert eth_px_data is not None, "ETH price data is empty"

bitmine_px_data = get_historical_close_px(bitmine_ticker, start_date=start_date, end_date=today)

In [ ]:
eth_px_data

In [ ]:
bitmine_info = yf.Ticker(bitmine_ticker)
outstanding_shares = bitmine_info.get_shares_full(start=start_date, end=today)

assert outstanding_shares is not None and len(outstanding_shares) > 0, "Outstanding shares data is empty"

outstanding_shares.index = pd.to_datetime(outstanding_shares.index)
if outstanding_shares.index.tz is not None:
    outstanding_shares.index = outstanding_shares.index.tz_localize(None)

outstanding_shares = outstanding_shares[~outstanding_shares.index.duplicated(keep='last')]
outstanding_shares

In [ ]:
bitmine_holdings['eth_px'] = bitmine_holdings['date'].apply(lambda x: get_px_for_date(x, eth_px_data))
bitmine_holdings['stock_px'] = bitmine_holdings['date'].apply(lambda x: get_px_for_date(x, bitmine_px_data))

In [ ]:
date_range = pd.date_range(start=start_date, end=today, freq='D')
daily_holdings = pd.DataFrame({'date': date_range})

daily_holdings = daily_holdings.merge(bitmine_holdings, on='date', how='left')
daily_holdings['eth'] = daily_holdings['eth'].ffill().round(0)

daily_holdings['shares_outstanding'] = daily_holdings['date'].apply(lambda x: get_px_for_date(x, outstanding_shares))
daily_holdings['eth_px'] = daily_holdings['date'].apply(lambda x: get_px_for_date(x, eth_px_data))
daily_holdings['stock_px'] = daily_holdings['date'].apply(lambda x: get_px_for_date(x, bitmine_px_data))

daily_holdings['nav'] = ((daily_holdings['eth'] * daily_holdings['eth_px']) / 1_000_000).round(4)
daily_holdings['market_cap_in_mil'] = ((daily_holdings['stock_px'] * daily_holdings['shares_outstanding']) / 1_000_000).round(4)
daily_holdings['eth_per_share'] = (daily_holdings['eth'] / daily_holdings['shares_outstanding']).round(6)
daily_holdings['nav_per_share'] = ((daily_holdings['eth'] * daily_holdings['eth_px']) / daily_holdings['shares_outstanding']).round(4)
daily_holdings['mnav'] = daily_holdings['stock_px'] / daily_holdings['nav_per_share']

daily_holdings = daily_holdings.sort_values('date')
result = daily_holdings[['date', 'eth', 'eth_px', 'nav', 'stock_px', 'shares_outstanding', 
                'market_cap_in_mil', 'eth_per_share', 'nav_per_share', 'mnav']].reset_index(drop=True)
result

In [ ]:
result[result['date'] >= pd.Timestamp('2025-07-08')]

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots with 4 rows and 2 columns
fig = make_subplots(rows=4, cols=2, 
                    subplot_titles=('ETH Holdings', 'Shares Outstanding',
                                  'Stock Price', 'NAV',
                                  'Market Cap (Millions)', 'ETH per Share',
                                  'NAV per Share', 'MNAV (Market Price/NAV)'))

# ETH Holdings
fig.add_trace(go.Scatter(x=result['date'], y=result['eth'], name='ETH Holdings'),
              row=1, col=1)

# Shares Outstanding
fig.add_trace(go.Scatter(x=result['date'], y=result['shares_outstanding'], name='Shares Outstanding'),
              row=1, col=2)

# Stock Price
fig.add_trace(go.Scatter(x=result['date'], y=result['stock_px'], name='Stock Price'),
              row=2, col=1)

# NAV
fig.add_trace(go.Scatter(x=result['date'], y=result['nav'], name='NAV (Millions)'),
              row=2, col=2)

# Market Cap
fig.add_trace(go.Scatter(x=result['date'], y=result['market_cap_in_mil'], name='Market Cap (Millions)'),
              row=3, col=1)

# ETH per Share
fig.add_trace(go.Scatter(x=result['date'], y=result['eth_per_share'], name='ETH per Share'),
              row=3, col=2)

# NAV per Share
fig.add_trace(go.Scatter(x=result['date'], y=result['nav_per_share'], name='NAV per Share'),
              row=4, col=1)

# MNAV
fig.add_trace(go.Scatter(x=result['date'], y=result['mnav'], name='MNAV'),
              row=4, col=2)

# Update layout
fig.update_layout(height=1200, width=1000, title_text="BitMine Metrics Over Time",
                 showlegend=True)
fig.update_xaxes(title_text="Date")

# Update y-axis labels
fig.update_yaxes(title_text="ETH", row=1, col=1)
fig.update_yaxes(title_text="Shares", row=1, col=2)
fig.update_yaxes(title_text="USD", row=2, col=1)
fig.update_yaxes(title_text="USD Millions", row=2, col=2)
fig.update_yaxes(title_text="USD Millions", row=3, col=1)
fig.update_yaxes(title_text="ETH", row=3, col=2)
fig.update_yaxes(title_text="USD", row=4, col=1)
fig.update_yaxes(title_text="Ratio", row=4, col=2)

fig.show()

# Growth and Value Creation Analysis

We'll analyze:
1. Growth rates for key metrics
2. Dilution impact vs. value creation
3. Market pricing efficiency (MNAV trends)
4. Capital deployment efficiency

In [ ]:
# Calculate daily growth rates (handle the first day separately)
daily_holdings['eth_growth'] = daily_holdings['eth'].pct_change().fillna(0)
daily_holdings['shares_growth'] = daily_holdings['shares_outstanding'].pct_change().fillna(0)
daily_holdings['nav_growth'] = daily_holdings['nav'].pct_change().fillna(0)

# Calculate cumulative metrics
daily_holdings['cum_eth_growth'] = ((1 + daily_holdings['eth_growth']).cumprod() - 1) * 100  # Convert to percentage
daily_holdings['cum_shares_growth'] = ((1 + daily_holdings['shares_growth']).cumprod() - 1) * 100
daily_holdings['cum_nav_growth'] = ((1 + daily_holdings['nav_growth']).cumprod() - 1) * 100

# Calculate efficiency metrics with safety checks
daily_holdings['value_creation_ratio'] = np.where(
    daily_holdings['shares_growth'] != 0,
    daily_holdings['nav_growth'] / daily_holdings['shares_growth'].abs(),
    np.nan
)

daily_holdings['eth_acquisition_efficiency'] = np.where(
    daily_holdings['shares_growth'] != 0,
    daily_holdings['eth_growth'] / daily_holdings['shares_growth'].abs(),
    np.nan
)

# Calculate rolling averages for smoother visualization
window = 7  # 7-day rolling window
rolling_metrics = daily_holdings[['date', 'mnav', 'value_creation_ratio', 'eth_acquisition_efficiency']].copy()
rolling_metrics['mnav_ma'] = daily_holdings['mnav'].rolling(window=window, min_periods=1).mean()
rolling_metrics['value_creation_ma'] = daily_holdings['value_creation_ratio'].rolling(window=window, min_periods=1).mean()
rolling_metrics['eth_efficiency_ma'] = daily_holdings['eth_acquisition_efficiency'].rolling(window=window, min_periods=1).mean()

# Create visualization
fig = make_subplots(rows=3, cols=1,
                    subplot_titles=('Cumulative Growth Comparison (%)',
                                  'Value Creation vs. Dilution',
                                  'Market Pricing Efficiency (MNAV)'),
                    vertical_spacing=0.15,
                    row_heights=[0.4, 0.3, 0.3])

# Cumulative Growth Comparison
fig.add_trace(go.Scatter(x=daily_holdings['date'], y=daily_holdings['cum_eth_growth'], 
                        name='ETH Holdings Growth %', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=daily_holdings['date'], y=daily_holdings['cum_shares_growth'], 
                        name='Shares Outstanding Growth %', line=dict(color='red')), row=1, col=1)
fig.add_trace(go.Scatter(x=daily_holdings['date'], y=daily_holdings['cum_nav_growth'], 
                        name='NAV Growth %', line=dict(color='green')), row=1, col=1)

# Value Creation vs. Dilution (only plot non-NaN values)
mask = rolling_metrics['value_creation_ma'].notna()
fig.add_trace(go.Scatter(x=rolling_metrics.loc[mask, 'date'], 
                        y=rolling_metrics.loc[mask, 'value_creation_ma'],
                        name='Value Creation Ratio (7d MA)', 
                        line=dict(color='purple')), row=2, col=1)

mask = rolling_metrics['eth_efficiency_ma'].notna()
fig.add_trace(go.Scatter(x=rolling_metrics.loc[mask, 'date'], 
                        y=rolling_metrics.loc[mask, 'eth_efficiency_ma'],
                        name='ETH Acquisition Efficiency (7d MA)', 
                        line=dict(color='orange')), row=2, col=1)

# MNAV Trend
fig.add_trace(go.Scatter(x=rolling_metrics['date'], y=rolling_metrics['mnav_ma'], 
                        name='MNAV (7d MA)', line=dict(color='brown')), row=3, col=1)
fig.add_hline(y=1, line_dash="dash", line_color="gray", row=3, col=1)

# Update layout
fig.update_layout(height=1000, width=1000, 
                 title_text="BitMine Growth and Efficiency Analysis",
                 showlegend=True)

# Update axes labels
fig.update_yaxes(title_text="Growth Rate (%)", row=1, col=1)
fig.update_yaxes(title_text="Ratio", row=2, col=1)
fig.update_yaxes(title_text="MNAV Ratio", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=1)

# Add reference line at y=0 for the value creation plot
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=2, col=1)

fig.show()

# Print summary statistics
print("\nSummary Statistics:")
print(f"Total ETH Growth: {daily_holdings['cum_eth_growth'].iloc[-1]:.1f}%")
print(f"Total Shares Growth: {daily_holdings['cum_shares_growth'].iloc[-1]:.1f}%")
print(f"Total NAV Growth: {daily_holdings['cum_nav_growth'].iloc[-1]:.1f}%")
print(f"\nLatest MNAV: {rolling_metrics['mnav_ma'].iloc[-1]:.3f}")
print(f"Latest Value Creation Ratio: {rolling_metrics['value_creation_ma'].iloc[-1]:.3f}")
print(f"Latest ETH Acquisition Efficiency: {rolling_metrics['eth_efficiency_ma'].iloc[-1]:.3f}")

# Analysis Interpretation

1. **Cumulative Growth Comparison**:
   - Shows relative growth rates of ETH holdings vs. shares outstanding vs. NAV
   - If NAV growth > shares growth, value is being created despite dilution
   - If ETH growth > shares growth, acquisition is efficient

2. **Value Creation vs. Dilution**:
   - Value Creation Ratio > 1 means NAV is growing faster than dilution
   - ETH Acquisition Efficiency > 1 means ETH holdings are growing faster than share dilution
   - Higher ratios indicate more efficient capital deployment

3. **MNAV (Market Price/NAV)**:
   - MNAV > 1 indicates market premium to NAV
   - MNAV < 1 indicates market discount to NAV
   - Trend shows market's evolving view of the strategy

Look for:
- Sustained value creation ratio > 1
- Stable or improving ETH acquisition efficiency
- MNAV trending toward or above 1

In [ ]:
# Define announcement dates
announcements = {
    'First ETH Purchase': pd.Timestamp('2025-07-24'),
    'Expansion Plan': pd.Timestamp('2025-08-12')
}

# Function to analyze price movements around dates
def analyze_price_movement(data, date, days_before=5, days_after=5):
    start_date = date - pd.Timedelta(days=days_before)
    end_date = date + pd.Timedelta(days=days_after)
    period_data = data[(data['date'] >= start_date) & (data['date'] <= end_date)].copy()
    
    if len(period_data) > 0:
        base_price = period_data.loc[period_data['date'] <= date, 'stock_px'].iloc[-1]
        period_data['price_change'] = (period_data['stock_px'] / base_price - 1) * 100
        return period_data
    return None

# Create subplots for announcement analysis
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('Stock Price Around Announcements',
                                  'MNAV Around Announcements',
                                  'Cumulative Returns Around Announcements',
                                  'Trading Volume Analysis'),
                    vertical_spacing=0.15,
                    horizontal_spacing=0.1)

# Analyze each announcement
colors = ['blue', 'red']
for i, (name, date) in enumerate(announcements.items()):
    period_data = analyze_price_movement(daily_holdings, date)
    
    if period_data is not None:
        # Stock Price
        fig.add_trace(go.Scatter(
            x=period_data['date'],
            y=period_data['stock_px'],
            name=f'{name} - Price',
            line=dict(color=colors[i]),
            showlegend=True
        ), row=1, col=1)
        
        # MNAV
        fig.add_trace(go.Scatter(
            x=period_data['date'],
            y=period_data['mnav'],
            name=f'{name} - MNAV',
            line=dict(color=colors[i], dash='dot'),
            showlegend=True
        ), row=1, col=2)
        
        # Cumulative Returns
        fig.add_trace(go.Scatter(
            x=period_data['date'],
            y=period_data['price_change'],
            name=f'{name} - Return',
            line=dict(color=colors[i]),
            showlegend=True
        ), row=2, col=1)
        
        # Add vertical lines for announcement dates
        for row in [1, 2]:
            for col in [1, 2]:
                fig.add_vline(x=date, line_dash="dash", line_color=colors[i],
                            row=row, col=col, opacity=0.3)

# Update layout
fig.update_layout(height=800, width=1200,
                 title_text="Analysis of Price Movement Around Key Announcements")

fig.update_yaxes(title_text="Stock Price (USD)", row=1, col=1)
fig.update_yaxes(title_text="MNAV Ratio", row=1, col=2)
fig.update_yaxes(title_text="Cumulative Return (%)", row=2, col=1)

fig.show()

# Print detailed analysis
print("\nDetailed Analysis of Announcements:")
for name, date in announcements.items():
    period_data = analyze_price_movement(daily_holdings, date)
    if period_data is not None:
        pre_price = period_data.loc[period_data['date'] < date, 'stock_px'].iloc[0]
        announcement_price = period_data.loc[period_data['date'] == date, 'stock_px'].iloc[0]
        post_price = period_data.loc[period_data['date'] > date, 'stock_px'].iloc[-1]
        
        pre_mnav = period_data.loc[period_data['date'] < date, 'mnav'].iloc[0]
        announcement_mnav = period_data.loc[period_data['date'] == date, 'mnav'].iloc[0]
        post_mnav = period_data.loc[period_data['date'] > date, 'mnav'].iloc[-1]
        
        print(f"\n{name} ({date.strftime('%Y-%m-%d')}):")
        print(f"Price Movement:")
        print(f"- Pre-announcement: ${pre_price:.2f}")
        print(f"- Announcement day: ${announcement_price:.2f} ({((announcement_price/pre_price - 1) * 100):.1f}% change)")
        print(f"- Post-announcement: ${post_price:.2f} ({((post_price/pre_price - 1) * 100):.1f}% total change)")
        
        print(f"MNAV Movement:")
        print(f"- Pre-announcement: {pre_mnav:.3f}")
        print(f"- Announcement day: {announcement_mnav:.3f}")
        print(f"- Post-announcement: {post_mnav:.3f}")

# Key Observations Around Announcements

1. **First ETH Purchase (July 24, 2025)**:
   - Represents the initial $2B ETH purchase announcement
   - Watch for:
     - Initial market reaction to the strategy
     - MNAV changes as market prices in the ETH holdings
     - Any premium/discount to NAV

2. **Expansion Plan (August 12, 2025)**:
   - Represents the $20B ETH purchase plan announcement
   - Key points:
     - Market's reaction to aggressive expansion
     - Changes in MNAV reflecting market confidence
     - Price movement relative to NAV growth

3. **Impact Analysis**:
   - Compare pre and post-announcement price levels
   - Look for changes in MNAV premium/discount
   - Assess market's efficiency in pricing the news